# Multi-Graphic Augmentation

In this experiment, I compare different methods on how they perform on graphics formed by combining other graphics. 

## Load Data and Model

In [ ]:
from vectorrvnn.interfaces import *
from vectorrvnn.trainutils import *
from vectorrvnn.data import *
from vectorrvnn.utils import *
from vectorrvnn.geometry import *
from vectorrvnn.baselines import *
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
from copy import deepcopy
from sklearn.manifold import TSNE

opts = Options().parse(testing=[
    '--checkpoints_dir', '../results',
    '--dataroot', '../data/All',
    '--embedding_size', '32', 
    '--load_ckpt', 'onebranch_oneof/best_0-796-07-13-2021-10-20-04.pth',                          
    '--modelcls', 'OneBranch',
    '--name', 'best_0-796-07-13-2021-10-20-04.pth',
    '--samplercls', 'DiscriminativeSampler',
    '--phase', 'test'
])


In [ ]:
model = buildModel(opts)
data = buildData(opts)
trainData, valData, trainDataLoader, valDataLoader = data
aug = GraphicCompose()
print(model.training)

## Some examples

In [ ]:
# Show some examples:
with torch.no_grad() :
    for i in range(10) : 
        print(i)
        graphic = aug(trng.choice(valData), valData)
        plt.imshow(rasterize(graphic.doc, 200, 200))
        plt.show()
        t = model.greedyTree(graphic)
        t.doc = graphic.doc
        treeImageFromGraph(t)
        plt.show()

## Comparing different eval metrics on all methods

In [ ]:
with torch.no_grad() :
    graphics = [aug(trng.choice(valData), valData) for _ in range(100)]
    ourTree = [model.greedyTree(_) for _ in graphics]
    sugT = [suggero(g.doc) for g in graphics]
    autoT = [autogroup(g.doc) for g in graphics]

In [ ]:
print('Ours      (FMI-1) = ', avg([fmi(g, t, 1) for g, t in zip(graphics, ourTree)]))
print('Suggero   (FMI-1) = ', avg([fmi(g, t, 1) for g, t in zip(graphics, sugT)]))
print('Autogroup (FMI-1) = ', avg([fmi(g, t, 1) for g, t in zip(graphics, autoT)]))
print("_________________________________________________________")
print('Ours      (FMI-2) = ', avg([fmi(g, t, 2) for g, t in zip(graphics, ourTree)]))
print('Suggero   (FMI-2) = ', avg([fmi(g, t, 2) for g, t in zip(graphics, sugT)]))
print('Autogroup (FMI-2) = ', avg([fmi(g, t, 2) for g, t in zip(graphics, autoT)]))
print("_________________________________________________________")
print('Ours      (FMI-3) = ', avg([fmi(g, t, 3) for g, t in zip(graphics, ourTree)]))
print('Suggero   (FMI-3) = ', avg([fmi(g, t, 3) for g, t in zip(graphics, sugT)]))
print('Autogroup (FMI-3) = ', avg([fmi(g, t, 3) for g, t in zip(graphics, autoT)]))
print("_________________________________________________________")
print('Ours      (cted) = ', avg([cted(g, t) / (g.number_of_nodes() + t.number_of_nodes()) for g, t in zip(graphics, ourTree)]))
print('Suggero   (cted) = ', avg([cted(g, t) / (g.number_of_nodes() + t.number_of_nodes()) for g, t in zip(graphics, sugT)]))
print('Autogroup (cted) = ', avg([cted(g, t) / (g.number_of_nodes() + t.number_of_nodes()) for g, t in zip(graphics, autoT)]))